In [ ]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import string
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from transformers import BertTokenizer, TFBertModel
from transformers import RobertaTokenizer, TFRobertaModel

In [ ]:
# Assuming 'Text' contains two sentences separated by '\n'

data[['Sentence1', 'Sentence2']] = data['Text'].str.split('\n', expand=True)

# Split the labeled dataset into training and validation sets

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
# Load Universal Sentence Encoder(USE) model from TensorFlow Hub

use_model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_embed = hub.load(use_model_url)


In [ ]:
# Encode sentences using USE

train_data['Embed1'] = use_embed(train_data['Sentence1'].tolist()).numpy().tolist()
train_data['Embed2'] = use_embed(train_data['Sentence2'].tolist()).numpy().tolist()

val_data['Embed1'] = use_embed(val_data['Sentence1'].tolist()).numpy().tolist()
val_data['Embed2'] = use_embed(val_data['Sentence2'].tolist()).numpy().tolist()


In [ ]:
# Siamese Network 

input1 = tf.keras.layers.Input(shape=(512,))
input2 = tf.keras.layers.Input(shape=(512,))
merged = tf.keras.layers.Lambda(lambda x: tf.norm(x[0] - x[1], axis=-1))(inputs=[input1, input2])

reshaped = tf.keras.layers.Reshape((1,))(merged)
output = tf.keras.layers.Dense(1, activation='sigmoid')(reshaped)

siamese_model = tf.keras.models.Model(inputs=[input1, input2], outputs=output)
siamese_model.compile(optimizer='adam', loss='mse')


In [ ]:

X_train1 = tf.constant(train_data['Embed1'].tolist())
X_train2 = tf.constant(train_data['Embed2'].tolist())
y_train = train_data['Score'].values

X_val1 = tf.constant(val_data['Embed1'].tolist())
X_val2 = tf.constant(val_data['Embed2'].tolist())
y_val = val_data['Score'].values


In [ ]:
# Train the Siamese Network

siamese_model.fit([X_train1, X_train2], y_train, epochs=3, batch_size=8, validation_data=([X_val1, X_val2], y_val))

In [ ]:
# Predictions

val_data['Predicted_Score'] = siamese_model.predict([X_val1, X_val2])